# Kopp et al 2021 Evaluation 
**Authorship:**
Adam Klie, *08/12/2022*
***
**Description:**
Notebook to perform a brief evaluation of trained models on the Kopp et al (2021) dataset.
***

In [7]:
# General imports
import os
import sys
import glob
import torch
import numpy as np

# EUGENe imports and settings
import eugene as eu
from eugene import dataload as dl
from eugene import models
from eugene import evaluate
from eugene import settings
settings.dataset_dir = "/cellar/users/aklie/data/eugene/revision/kopp21"
settings.output_dir = "/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/output/revision/kopp21"
settings.logging_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/logs/kopp21"
settings.config_dir = "/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/kopp21"

# EUGENe packages
import seqdata as sd

# For illustrator editing
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Print versions
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Eugene version: {eu.__version__}")
print(f"SeqData version: {sd.__version__}")
print(f"PyTorch version: {torch.__version__}")

Python version: 3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:39:03) 
[GCC 11.3.0]
NumPy version: 1.23.5
Eugene version: 0.0.8
SeqData version: 0.0.1
PyTorch version: 2.0.0


# Load in the test `SeqData`(s)

In [4]:
sdata = sd.open_zarr(os.path.join(settings.dataset_dir, 'kopp21_test.zarr'))
sdata

<xarray.Dataset>
Dimensions:     (_sequence: 64309, cov_sample: 1, _length: 500, length: 500,
                 _ohe: 4)
Coordinates:
  * cov_sample  (cov_sample) object 'stam'
Dimensions without coordinates: _sequence, _length, length, _ohe
Data variables:
    chrom       (_sequence) object dask.array<chunksize=(32155,), meta=np.ndarray>
    chromEnd    (_sequence) int64 dask.array<chunksize=(32155,), meta=np.ndarray>
    chromStart  (_sequence) int64 dask.array<chunksize=(32155,), meta=np.ndarray>
    cov         (_sequence, cov_sample, _length) uint16 dask.array<chunksize=(8039, 1, 63), meta=np.ndarray>
    ohe_seq     (_sequence, length, _ohe) uint8 dask.array<chunksize=(8039, 125, 1), meta=np.ndarray>
    seq         (_sequence, _length) |S1 dask.array<chunksize=(8039, 125), meta=np.ndarray>
    strand      (_sequence) object dask.array<chunksize=(32155,), meta=np.ndarray>
    target      (_sequence) uint8 dask.array<chunksize=(64309,), meta=np.ndarray>
    train_test  (_sequence) bool dask.array<chunksize=(64309,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    0
    sequence_dim:  _sequence

# Get test set predictions for each model

In [9]:
from itertools import product
configs = ["fcn.yaml", "cnn.yaml", "hybrid.yaml", "kopp21_cnn.yaml"]
trials = 1

for config, trial in product(configs, range(1, trials+1)):
    model_name = config.split('.')[0]
    print(model_name)

    # Load the model
    model_file = glob.glob(os.path.join(settings.logging_dir, model_name, f"trial_{trial}", "checkpoints", "*"))[0]
    model = models.load_config(config_path=config)
    best_model = models.SequenceModule.load_from_checkpoint(model_file, arch=model.arch)
    
    # Set-up transforms
    transforms = {
        "target": lambda x: torch.tensor(x, dtype=torch.float32),
        "ohe_seq": lambda x: torch.tensor(x, dtype=torch.float32).swapaxes(1, 2)
    }
    
    # Evaluate
    evaluate.predictions_sequence_module(
        model=best_model,
        sdata=sdata,
        seq_key="ohe_seq",
        target_keys="target",
        gpus=1,
        batch_size=2048,
        num_workers=4,
        prefetch_factor=2,
        in_memory=True,
        transforms=transforms,
        file_label="test",
        name=model_name,
        version=f"trial_{trial}",
        prefix=f"{model_name}_trial_{trial}_"
    )

# Save the predictions
pred_keys = [k for k in sdata.data_vars.keys() if "predictions" in k]
(
    sdata[['chrom', 'chromStart', 'chromEnd', 'target', *pred_keys]]
    .to_dataframe()
    .to_csv(os.path.join(settings.output_dir, f"kopp21_test_predictions.tsv"), sep="\t", index=False)
)
sd.to_zarr(sdata, os.path.join(settings.output_dir, f"kopp21_test_predictions.zarr"), mode="w", load_first=True)

/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.3, which is newer than your current Lightning version: v2.0.0
  rank_zero_warn(
[rank: 0] Global seed set to 1


fcn
Loading ohe_seq and ['target'] into memory


/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_utilities/core/imports.py:116: UserWarning: Unbuilt egg for seqpro [unknown version] (/cellar/users/aklie/projects/ML4GLand/SeqPro)
  pkg_resources.require(self.requirement)
/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available o

Predicting: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.3, which is newer than your current Lightning version: v2.0.0
  rank_zero_warn(
[rank: 0] Global seed set to 1


...

/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.3, which is newer than your current Lightning version: v2.0.0
  rank_zero_warn(
[rank: 0] Global seed set to 1


kopp21_cnn
Loading ohe_seq and ['target'] into memory


/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

OSError: Cannot save file into a non-existent directory: '/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/output/revision/kopp21/test'

# DONE!

---

# Scratch